In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from keras import metrics



%matplotlib inline

In [2]:
# Load the dataset
df = pd.read_excel(r'/FRAUD/ccfraudTrain.xlsx')

df1 = pd.read_excel(r'/FRAUD/ccfraudTest.xlsx')
# For snakey diagram
#df1 = pd.read_excel(r'C:\Users\sg_cl\Desktop\masters subjects\DSRT 736\Fraud Dataset\New_data_CCFD\Sankey_data.xlsx')

In [7]:
df.columns

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'merch_lat',
       'merch_long', 'is_fraud', 'trans_date_trans_time_HR'],
      dtype='object')

In [58]:
#print (df['trans_date_trans_time'].head())
from datetime import datetime as dt
import pytz  # Import the pytz module for time zones

df['trans_date_trans_time_HR'] = df['trans_date_trans_time'].dt.strftime("%H")

print (df.head())

df1['trans_date_trans_time_HR'] = df1['trans_date_trans_time'].dt.strftime("%H")

print (df1.head())

df['trans_date_trans_time_HR'] = df['trans_date_trans_time_HR'].astype(int)

df1['trans_date_trans_time_HR'] = df1['trans_date_trans_time_HR'].astype(int)

df['trans_day_of_week'] = df['trans_date_trans_time'].dt.dayofweek

df1['trans_day_of_week'] = df1['trans_date_trans_time'].dt.dayofweek

   Unnamed: 0 trans_date_trans_time    cc_num  merchant  category       amt  \
0           0   2019-01-01 00:00:18 -0.316664       514         8 -0.408305   
1           1   2019-01-01 00:00:44 -0.318729       241         4  0.231013   
2           2   2019-01-01 00:00:51 -0.318700       390         0  0.936726   
3           3   2019-01-01 00:01:16 -0.316029       360         2 -0.158042   
4           4   2019-01-01 00:03:06 -0.318443       297         9 -0.177048   

   gender  city  state       zip       lat      long  city_pop  job  \
0       0   517     27 -0.749036 -0.483482  0.657638 -0.282913  370   
1       0   600     47  1.872197  2.039539 -2.033949 -0.293977  428   
2       1   459     13  1.280778  0.718445 -1.601601 -0.280734  307   
3       1    83     26  0.402647  1.516144 -1.590829 -0.288058  328   
4       1   212     45 -0.905962 -0.022192  0.782302 -0.294142  116   

         dob  merch_lat  merch_long  is_fraud trans_date_trans_time_HR  \
0 1988-03-09  -0.493457 

In [79]:
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['trans_date'] = df['trans_date_trans_time'].dt.date

In [87]:
df1['trans_date_trans_time'] = pd.to_datetime(df1['trans_date_trans_time'])
df1['trans_date'] = df1['trans_date_trans_time'].dt.date

In [105]:
#df = df.drop('unix_time', axis=1)
df1 = df1.drop('unix_time', axis=1)

df = df.drop('first', axis=1)
df1 = df1.drop('first', axis=1)

df = df.drop('last', axis=1)
df1 = df1.drop('last', axis=1)

df = df.drop('street', axis=1)
df1 = df1.drop('street', axis=1)

df = df.drop('trans_num', axis=1)
df1 = df1.drop('trans_num', axis=1)

df = df.drop('trans_date_trans_time', axis=1)
df1 = df1.drop('trans_date_trans_time', axis=1)


df = df.drop('trans_date', axis=1)
df1 = df1.drop('trans_date', axis=1)


df = df.drop('dob', axis=1)
df1 = df1.drop('dob', axis=1)




In [106]:
# Preprocess your data
# This is a placeholder. You need to fill in with your own preprocessing steps.
X = df.drop('is_fraud', axis=1)
y = df['is_fraud']

In [107]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [108]:
# Print the shapes of the resulting subsets
print("Training set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)

Training set shape: (838860, 17) (838860,)
Test set shape: (209715, 17) (209715,)


In [124]:
# Reshape the data to be 3D to work with CNN
X_train = np.array(X_train)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = np.array(X_test)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_train = np.array(X_train)
y_train = np.array(y_train)
y_train = tf.cast(y_train, tf.float32)


y_test = y_test.astype('float32')

In [125]:
# Print the shapes of the resulting subsets
print("Training set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)

Training set shape: (838860, 17, 1) (838860,)
Test set shape: (209715, 17, 1) (209715,)


In [126]:
X_train[1:3]

array([[[ 1.20092000e+05],
        [-3.18566600e-01],
        [ 5.43000000e+02],
        [ 4.00000000e+00],
        [-8.17064759e-02],
        [ 1.00000000e+00],
        [ 6.80000000e+02],
        [ 5.00000000e+01],
        [ 1.25285802e+00],
        [ 8.86718262e-01],
        [-1.30654106e+00],
        [-2.30296689e-01],
        [ 2.66000000e+02],
        [ 1.06860747e+00],
        [-1.29805825e+00],
        [ 0.00000000e+00],
        [ 5.00000000e+00]],

       [[ 1.74845000e+05],
        [-3.14612260e-01],
        [ 6.75000000e+02],
        [ 6.00000000e+00],
        [-3.31281715e-01],
        [ 1.00000000e+00],
        [ 7.28000000e+02],
        [ 4.80000000e+01],
        [ 2.17577642e-01],
        [ 9.20459657e-01],
        [-5.19126645e-02],
        [-2.93963280e-01],
        [ 3.81000000e+02],
        [ 9.99504925e-01],
        [-4.63814159e-02],
        [ 1.30000000e+01],
        [ 0.00000000e+00]]])

In [127]:
# Define the model
model = Sequential()
model.add(Conv1D(32,2,activation  = 'relu', input_shape=X_train[0].shape))
model.add(Flatten())
model.add(Dense(64, activation= 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [128]:
#Compile the model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [metrics.F1Score()] )

In [129]:
# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
26215/26215 [==============================] - 52s 2ms/step - loss: 13.1058 - f1_score: 0.0118 - val_loss: 0.0262 - val_f1_score: 0.0114
Epoch 2/10
26215/26215 [==============================] - 48s 2ms/step - loss: 0.1090 - f1_score: 0.0116 - val_loss: 0.0178 - val_f1_score: 0.0114
Epoch 3/10
26215/26215 [==============================] - 47s 2ms/step - loss: 0.0727 - f1_score: 0.0114 - val_loss: 0.0145 - val_f1_score: 0.0114
Epoch 4/10
26215/26215 [==============================] - 52s 2ms/step - loss: 0.0175 - f1_score: 0.0114 - val_loss: 0.0183 - val_f1_score: 0.0114
Epoch 5/10
26215/26215 [==============================] - 48s 2ms/step - loss: 0.0140 - f1_score: 0.0114 - val_loss: 0.0176 - val_f1_score: 0.0114
Epoch 6/10
26215/26215 [==============================] - 49s 2ms/step - loss: 0.0132 - f1_score: 0.0114 - val_loss: 0.0126 - val_f1_score: 0.0114
Epoch 7/10
26215/26215 [==============================] - 52s 2ms/step - loss: 0.0126 - f1_score: 0.0114 - val_loss: 

In [42]:
print("Missing values:", df.isnull().sum())

Missing values: Unnamed: 0                  0
trans_date_trans_time       0
cc_num                      0
merchant                    0
category                    0
amt                         0
gender                      0
city                        0
state                       0
zip                         0
lat                         0
long                        0
city_pop                    0
job                         0
dob                         0
trans_num                   0
merch_lat                   0
merch_long                  0
is_fraud                    0
trans_date_trans_time_HR    0
trans_day_of_week           0
dtype: int64


In [86]:
# checking the dataset
df.info()
df.describe()
df1.info()
df1.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 20 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   Unnamed: 0                1048575 non-null  int64         
 1   cc_num                    1048575 non-null  float64       
 2   merchant                  1048575 non-null  int64         
 3   category                  1048575 non-null  int64         
 4   amt                       1048575 non-null  float64       
 5   gender                    1048575 non-null  int64         
 6   city                      1048575 non-null  int64         
 7   state                     1048575 non-null  int64         
 8   zip                       1048575 non-null  float64       
 9   lat                       1048575 non-null  float64       
 10  long                      1048575 non-null  float64       
 11  city_pop                  1048575 non-null  float6

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,merch_lat,merch_long,is_fraud,trans_date_trans_time_HR,trans_day_of_week
count,555719.000000,5.557190e+05,555719.000000,555719.000000,555719.000000,555719.000000,5.557190e+05,555719.000000,555719.000000,555719.000000,555719.000000,555719.000000
mean,277859.000000,4.178387e+17,69.392810,48842.628015,38.543253,-90.231325,8.822189e+04,38.542798,-90.231380,0.003860,12.809062,2.726779
std,160422.401459,1.309837e+18,156.745941,26855.283328,5.061336,13.721780,3.003909e+05,5.095829,13.733071,0.062008,6.810924,2.178681
min,0.000000,6.041621e+10,1.000000,1257.000000,20.027100,-165.672300,2.300000e+01,19.027422,-166.671575,0.000000,0.000000,0.000000
25%,138929.500000,1.800429e+14,9.630000,26292.000000,34.668900,-96.798000,7.410000e+02,34.755301,-96.905129,0.000000,7.000000,1.000000
50%,277859.000000,3.521417e+15,47.290000,48174.000000,39.371600,-87.476900,2.408000e+03,39.376593,-87.445204,0.000000,14.000000,2.000000
75%,416788.500000,4.635331e+15,83.010000,72011.000000,41.894800,-80.175200,1.968500e+04,41.954163,-80.264637,0.000000,19.000000,5.000000
max,555718.000000,4.992346e+18,22768.110000,99921.000000,65.689900,-67.950300,2.906700e+06,66.679297,-66.952026,1.000000,23.000000,6.000000


In [ ]:
# Select the desired columns and perform aggregation on the subset
subset_df = df[[ 'cc_num', 'trans_date_trans_time_HR', 'merchant', 'category', 'gender','state','zip','city_pop','lat', 'long','merch_lat',
       'merch_long', 'amt','weight','is_fraud']].reset_index()
subset_df1 = df1[[ 'cc_num', 'trans_date_trans_time_HR', 'merchant', 'category', 'gender','state','zip','city_pop','lat', 'long','merch_lat',
       'merch_long', 'amt','is_fraud']].reset_index()

In [47]:
# Scale the Numeric column
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df['amt'] = scaler.fit_transform(df['amt'].values.reshape(-1, 1))
df['cc_num'] = scaler.fit_transform(df['cc_num'].values.reshape(-1, 1))
df['zip'] = scaler.fit_transform(df['zip'].values.reshape(-1, 1))
df['lat'] = scaler.fit_transform(df['lat'].values.reshape(-1, 1))
df['long'] = scaler.fit_transform(df['long'].values.reshape(-1, 1))
df['city_pop'] = scaler.fit_transform(df['city_pop'].values.reshape(-1, 1))
df['merch_lat'] = scaler.fit_transform(df['merch_lat'].values.reshape(-1, 1))
df['merch_long'] = scaler.fit_transform(df['merch_long'].values.reshape(-1, 1))

In [48]:
# Label encode the Categorical column
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['gender'] = encoder.fit_transform(df['gender'])
df['merchant'] = encoder.fit_transform(df['merchant'])
df['category'] = encoder.fit_transform(df['category'])
df['city'] = encoder.fit_transform(df['city'])
df['state'] = encoder.fit_transform(df['state'])
df['job'] = encoder.fit_transform(df['job'])

# Extract features from 'trans_date_trans_time' Like did above

In [80]:
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,gender,city,state,zip,...,long,city_pop,job,dob,merch_lat,merch_long,is_fraud,trans_date_trans_time_HR,trans_day_of_week,trans_date
0,0,2019-01-01 00:00:18,-0.316664,514,8,-0.408305,0,517,27,-0.749036,...,0.657638,-0.282913,370,1988-03-09,-0.493457,0.593872,0,0,1,2019-01-01
1,1,2019-01-01 00:00:44,-0.318729,241,4,0.231013,0,600,47,1.872197,...,-2.033949,-0.293977,428,1978-06-21,2.078870,-2.030364,0,0,1,2019-01-01
2,2,2019-01-01 00:00:51,-0.318700,390,0,0.936726,1,459,13,1.280778,...,-1.601601,-0.280734,307,1962-01-19,0.903352,-1.592341,0,0,1,2019-01-01
3,3,2019-01-01 00:01:16,-0.316029,360,2,-0.158042,1,83,26,0.402647,...,-1.590829,-0.288058,328,1967-01-12,1.663175,-1.621866,0,0,1,2019-01-01
4,4,2019-01-01 00:03:06,-0.318443,297,9,-0.177048,1,212,45,-0.905962,...,0.782302,-0.294142,116,1986-03-28,0.027691,0.841920,0,0,1,2019-01-01
